This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'szszg-cAUKpGVoNESaRe'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [11]:
# First, import the relevant modules
import requests
import json


In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&limit=1&api_key=szszg-cAUKpGVoNESaRe")


In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2019-11-08',
  'frequency': 'daily',
  'data': [['2019-11-08',
    101.8,
    102.8,
    100.2,
    101.8,
    None,
    127304.0,
    12917154.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Task 1
Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [17]:
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=szszg-cAUKpGVoNESaRe")


### Task 2
Convert the returned JSON object into a Python dictionary.

In [18]:
r_dict = r.json()

### Task 3
Calculate what the highest and lowest opening prices were for the stock in this period.

In [24]:
# Using python dictionary comprehension
r_dict_opening = {row[0]: row[1] for row in r_dict["dataset_data"]["data"] if row[1] != None}
min_opening = min(r_dict_opening.items(), key=lambda k: k[1])
max_opening = max(r_dict_opening.items(), key=lambda k: k[1])


print("The highest opening price in this period was ${} per share.".format(max_opening[1]) )
print("The lowest opening price in this period was ${} per share.".format(min_opening[1]) )

The highest opening price was $53.11 per share
The lowest opening price was $34.0 per share


### Task 4
What was the largest change in any one day (based on High and Low price)?

In [26]:
# Using python dictionary comprehension
r_dict_change = {row[0]: row[2] - row[3] for row in r_dict["dataset_data"]["data"]}
max_change = max(r_dict_change.items(), key=lambda k: k[1])

print("The largest change in any one day was ${} per share".format(max_change[1]) )

The largest change in any one day was $2.8100000000000023 per share


### Task 5
What was the largest change between any two days (based on Closing Price)?

In [29]:
# Define first day and initialise empty dict
r_two_day_changes = {}
r_closing_first_day =  r_dict["dataset_data"]["data"][0][4]

for row in r_dict["dataset_data"]["data"][1:]:
    r_closing = row[4]
    r_two_day_changes[row[0]] = abs(r_closing_first_day - r_closing) # use abs to avoid negatives
    r_closing_first_day = r_closing

max_two_day_change = max(r_two_day_changes.items(), key=lambda k:k[1])

print("The largest change between any two days was ${} per share".format(max_two_day_change[1]))


The largest change between any two days was $2.559999999999995 per share


### Task 6
What was the average daily trading volume during this year?

In [33]:
# For looping over all data of year
r_trading_sum=0
for  i in range(len(r_dict["dataset_data"]["data"])):
    r_trading_sum+=r_dict["dataset_data"]["data"][i][6]
    
avg_daily_volume = r_trading_sum/len(r_dict["dataset_data"]["data"])

print("The average daily trading volume during this year was $"+ str(avg_daily_volume))

The average daily trading volume during this year was $89124.33725490196


### Task 7
What was the median trading volume during this year.

In [39]:
# Create sorted list of daily trading volumes
trade_volume = sorted([row[6] for row in r_dict["dataset_data"]["data"]])


def median(x): #Function takes already sorted lists
    length = len(x)
    index = (length - 1) // 2
    
    if (length % 2):     # Check if odd or even number of items in list and find median for each case
        return x[index]
    else:
        return (x[index] + x[index + 1])/2.0
   
print("The median trading volume during this year was $"+ str(median(trade_volume)))

The median trading volume during this year was $76286.0
